# Euclid AAS 247 Workshop -- 1b. Catalog Access

## Learning Goals

By the end of this tutorial, you will: 
- Understand the basic characteristics of Euclid Q1 MER catalogs.
- What columns are available in the MER catalog.
- How to query with ADQL in the MER catalog.

## Introduction

Euclid launched in July 2023 as a European Space Agency (ESA) mission with involvement by NASA. The primary science goals of Euclid are to better understand the composition and evolution of the dark Universe. The Euclid mission is providing space-based imaging and spectroscopy as well as supporting ground-based imaging to achieve these primary goals. These data will be archived by multiple global repositories, including IRSA, where they will support transformational work in many areas of astrophysics.

Euclid Quick Release 1 (Q1) consists of consists of ~30 TB of imaging, spectroscopy, and catalogs covering four non-contiguous fields: Euclid Deep Field North (22.9 sq deg), Euclid Deep Field Fornax (12.1 sq deg), Euclid Deep Field South (28.1 sq deg), and LDN1641.

Each entry in the MER catalog is a single source containing all its photometry from the MER Mosaics (VIS, Y, J, H and any accompanying external ground observations) along with other basic measurements, like size and shape. This notebook provides an introduction to the MER catalog released as part of Euclid Q1. Other Euclid notebooks show how to use other data products released as part of Euclid Q1.

## Data volume

Each MER image is approximately 1.47 GB. Downloading can take some time.

## Imports

In [3]:
# Uncomment the next line to install dependencies if needed.
# !pip install numpy matplotlib 'astroquery>=0.4.10'

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from astroquery.ipac.irsa import Irsa

# Introduction to Euclid Q1 MER catalog

## 1. Download MER catalog from IRSA directly to this notebook

First, have a look at what Euclid catalogs are available. With the list_catalogs functionality, we’ll receive a list of the name of the catalogs as well as their brief desciption.

In [2]:
Irsa.list_catalogs(filter='euclid')

{'euclid.tileid_association_q1': 'Euclid Q1 TILEID to Observation ID Association Table',
 'euclid.objectid_spectrafile_association_q1': 'Euclid Q1 Object ID to Spectral File Association Table',
 'euclid.observation_euclid_q1': 'Euclid Q1 CAOM Observation Table',
 'euclid.plane_euclid_q1': 'Euclid Q1 CAOM Plane Table',
 'euclid.artifact_euclid_q1': 'Euclid Q1 CAOM Artifact Table',
 'euclid_q1_mer_catalogue': 'Euclid Q1 MER Catalog',
 'euclid_q1_mer_morphology': 'Euclid Q1 MER Morphology',
 'euclid_q1_mer_cutouts': 'Euclid Q1 MER Cutouts',
 'euclid_q1_phz_photo_z': 'Euclid Q1 PHZ Photo-z Catalog',
 'euclid_q1_phz_star_sed': 'Euclid Q1 PHZ Star SED Catalog',
 'euclid_q1_phz_galaxy_sed': 'Euclid Q1 PHZ Galaxy SED Catalog',
 'euclid_q1_phz_classification': 'Euclid Q1 PHZ Classification Catalog',
 'euclid_q1_phz_qso_physical_parameters': 'Euclid Q1 PHZ QSO Physical Parameters Catalog',
 'euclid_q1_phz_nir_physical_parameters': 'Euclid Q1 PHZ NIR Physical Parameters Catalog',
 'euclid_q1_phz_

### Choose the Euclid MER table

In [3]:
table_mer= 'euclid_q1_mer_catalogue'

### Learn some information about the table:
- How many columns are there?
- List the column names

In [4]:
columns_info = Irsa.list_columns(catalog=table_mer)
print(len(columns_info))

477


In [5]:
columns_info

{'flux_g_ext_hsc_templfit': 'Gext HSC band source template fitting flux (TPHOT measurement)',
 'flux_z_ext_hsc_templfit': 'Zext HSC band source template fitting flux (TPHOT measurement)',
 'fluxerr_y_templfit': 'NIR Y band source template fitting flux error (TPHOT measurement)',
 'fluxerr_j_templfit': 'NIR J band source template fitting flux error (TPHOT measurement)',
 'fluxerr_h_templfit': 'NIR H band source template fitting flux error (TPHOT measurement)',
 'fluxerr_u_ext_decam_templfit': 'Uext DECam band source template fitting flux error (TPHOT measurement)',
 'fluxerr_g_ext_decam_templfit': 'Gext DECam band source template fitting flux error (TPHOT measurement)',
 'fluxerr_r_ext_decam_templfit': 'Rext DECam band source template fitting flux error (TPHOT measurement)',
 'fluxerr_i_ext_decam_templfit': 'Iext DECam band source template fitting flux error (TPHOT measurement)',
 'fluxerr_z_ext_decam_templfit': 'Zext DECam band source template fitting flux error (TPHOT measurement)',
 

### Define the following ADQL query to find the first 10k stars in the MER catalog

Since we are just using the MER catalog alone, it does not have a column for classification. We can use the point_like_flag = 1 or point_like_prob>0.99 for stars.

Set all the fluxes to be greater than 0 so the object is detected in all four Euclid MER mosaic images

In [6]:
adql_stars = f"SELECT TOP 10000 mer.ra, mer.dec, mer.flux_vis_psf, mer.fluxerr_vis_psf, mer.flux_y_templfit, \
mer.flux_j_templfit, mer.flux_h_templfit, mer.point_like_prob, mer.extended_prob \
FROM {table_mer} AS mer \
WHERE  mer.flux_vis_psf > 0 \
AND mer.flux_y_templfit > 0 \
AND mer.flux_j_templfit > 0 \
AND mer.flux_h_templfit > 0 \
AND mer.point_like_flag = 1  \
"


## Run the query
result_stars = Irsa.query_tap(adql_stars).to_table()


In [7]:
result_stars[:5]

ra,dec,flux_vis_psf,fluxerr_vis_psf,flux_y_templfit,flux_j_templfit,flux_h_templfit,point_like_prob,extended_prob
deg,deg,uJy,uJy,uJy,uJy,uJy,,
float64,float64,float32,float32,float32,float32,float32,float32,float32
274.41143605,65.92522229,18.06845093,0.05907947198,40.79600143,51.24997711,50.90077591,0.9834615588,--
274.82614525,65.91619631,20.08031464,0.05796840787,62.98400116,72.14587402,73.11473083,0.9726842046,--
275.36148169,65.91654475,9.404380798,0.05080717802,72.25162506,90.76555634,92.41596222,0.9804971218,--
275.15211165,65.91879386,8.089911461,0.029209584,31.106287,37.05817795,36.19960022,0.9785846472,--
274.68339103,65.91429329,33.57302475,0.1107933447,129.1610565,157.439621,164.0423431,0.9811390638,--


## About this Notebook

**Author(s)**: Tiffany Meshkat <br>
**Keyword(s)**: Euclid, Q1, AAS 247 Workshop <br>
**First published**: Dec 11, 2025 <br>
**Last updated**: Dec 11, 2025